In [ ]:
## Aim: For every locations, mark 500 km buffer circle around it, and.. 
## ..count the number fire event and total FRP from FIRMS MODIS firecount data happened within that buffer_circle.

In [1]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [2]:
from shapely.geometry import Polygon

In [3]:
# Shape file Firecount data

fp = r'C:/Users/LENOVO/Documents/Diwali_Impact_coding/Firecount/Country/MODIS_Country/fire_nrt_M-C61_332626.shp'
map_df = gpd.read_file(fp)

### Marking circle around a loaction (lat,lon) and its geometry data.

In [8]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

In [9]:
proj_wgs84

<Other Coordinate Operation Transformer: longlat>
Description: PROJ-based coordinate operation
Area of Use:
- undefined

In [13]:
## Function to create km circle around specified lat,lon

def buffer_circle(lat, lon, km):
    
    aeqp = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0' # AEQP-Azimuthal equidistant projection
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqp.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance 
    return transform(project, buf).exterior.coords[:]

In [53]:
### Uploading Site locations (lat,lon) data

Site_locations_data=pd.read_excel('DiwaliData_DataAllIndia_7D_Feb27.xlsx', sheet_name='Locations')

In [54]:
Lattitude=Site_locations_data['Lattitude']
Longitude=Site_locations_data['Longitude']

In [55]:
circle_pts=pd.DataFrame() # Dataframe to store circle lat,lon datapts.
ult_data=pd.DataFrame([]) # Final output data
for i in range(len(Site_locations_data)):
    
    #-----Function to create 500 km circle around lat,lon
    circle_pts =pd.DataFrame(buffer_circle(Lattitude[i],Longitude[i],500))  
    circle_pts.rename(columns = {0:'Longitude',1:'Latitude'}, inplace = True)
   
    lon_point_list=circle_pts['Longitude'] # Lon of formed buffer_circle.
    lat_point_list=circle_pts['Latitude'] # Lat of formed buffer_circle.
    
    #---Converting circular_pts (lat,lon) data as geometry polygon data.
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])
    
    #---------Dataframe to store details of Fire count data(map_df) which falls within polygon
    new_df=pd.DataFrame([]) # Dataframe to store firecount datapts 
    for j in range(len(map_df)): # iterating over firecount data to collect all points falling within polygon.
        Pt=Point(map_df['LONGITUDE'][j],map_df['LATITUDE'][j]) # Marking point from locations (lat,lon) details.
        if Pt.within(polygon).bool()==True: # Checking whether falling in polygon, True-if falling.
            new_df=pd.concat([new_df,map_df[j:j+1]])
            
    # Extracting necessary/required details from new_df after done for particular location.
    temp=pd.DataFrame() # Dataframe to store temporarily details of every locations.
    temp['FRP']=[new_df['FRP'].sum()] # FRP data
    temp['Latitude']=Site_locations_data['City'][i] # locations data
    temp['Count']=[new_df['FRP'].count()] # Count data
    ult_data=pd.concat([ult_data,temp])  # Concatenating details of every locations, taking latest from 'temp'.

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

In [59]:
ult_data # City and the total FRP and fireevent Count happened within 500 km.

,FRP,Latitude,Count
0,1337.78,Amaravati,95
0,2214.17,Anantpur,160
0,1543.92,Tirupati,110
0,1319.49,Vijayawada,94
0,1458.62,Vishakhapatnam,122
...,...,...,...
0,3632.14,Ratlam,225
0,3689.39,Ujjain,226
0,2570.02,Bhilai,208
0,2801.27,Bilaspur,232
